In [3]:
# | default_exp backend.tools.code_interpreter

In [2]:
# | export
from agentic.backend.tools.base import Tool
from agentic.backend.schemas import CodeInterpreterParams
import sys
import io
from contextlib import redirect_stdout, redirect_stderr
from typing import Dict, Any
import time

class CodeInterpreterTool(Tool):
    def get_tool_schema(self) -> Dict[str, Any]:
        """Return the OpenAI-compatible schema for code_interpreter."""
        return {
                "type": "function",
                "function": {
                    "name": "code_interpreter",
                    "description": "Execute Python code with visualization support. Handles plotting, data analysis, calculations, and complex computations with result capture.",
                    "parameters": {
                        "type": "object",
                        "properties": {
                            "code": {"type": "string", "description": "Python code to execute"},
                            "language": {"type": "string", "enum": ["python"], "description": "Programming language"},
                            "capture_output": {"type": "boolean", "description": "Capture stdout/stderr"},
                            "save_plots": {"type": "boolean", "description": "Save matplotlib plots as images"},
                            "timeout": {"type": "integer", "description": "Execution timeout in seconds"},
                            "working_dir": {"type": "string", "description": "Working directory for execution"},
                            "requirements": {"type": "array", "items": {"type": "string"}, "description": "Required packages to install"}
                        },
                        "required": ["code"]
                    }
                }
            }

    def execute(self, **kwargs) -> Dict[str, Any]:
        """Execute Python code with output capture."""
        try:
            params = CodeInterpreterParams(**kwargs)
            
            if params.language != "python":
                return {"error": "Only Python is supported"}
            
            start_time = time.time()
            stdout = io.StringIO()
            stderr = io.StringIO()
            
            try:
                with redirect_stdout(stdout), redirect_stderr(stderr):
                    exec(params.code, {}, {})
                
                execution_time = time.time() - start_time
                return {
                    "success": True,
                    "stdout": stdout.getvalue(),
                    "stderr": stderr.getvalue(),
                    "execution_time": execution_time,
                    "plot_count": 0,  # No plotting support in this version
                    "message": "Code executed successfully"
                }
                
            except Exception as exec_error:
                return {
                    "success": False,
                    "error": str(exec_error),
                    "stdout": stdout.getvalue(),
                    "stderr": stderr.getvalue(),
                    "execution_time": time.time() - start_time
                }
                
        except Exception as e:
            return {"error": f"Invalid parameters: {str(e)}"}